## CPSC 477 Final Project Part 1: Augmenting ECTSum With Gemini


In [6]:
# !pip install google-generativeai
import google.generativeai as genai
from dotenv import load_dotenv
import os
env = load_dotenv(override = True)

In [8]:
genai.configure(api_key = os.environ["API_KEY"])
model = genai.GenerativeModel("gemini-pro")

In [10]:
prompt = "You are a financial advisor tasked with creating a short summary of an earnings call transcript. You only want to summarize or re-iterate points that would be relevant, critical, or informational to someone who wants to skim over the important details of a long transcript.\n\nBelow is an earnings call transcript. Please summarize this transcript in one paragraph. Keep the summary below 500 words. Include relevant information from the Earnings Call Transcript in your summary. Furthermore, it is very important that you incorporate all the information from the Key Points and spread it out throughout your summary."
transcript = ""
key_points = ""

with open("AAN_q1_2021.txt") as f:
    transcript = f.read().strip()
    
with open("AAN_q1_2021_summary.txt") as f:
    key_points = f.read().strip()
    
print(len(transcript))

response = model.generate_content(f"{prompt}\n\nEarnings Call Transcript:\n{transcript}\n\nKey Points:\n{key_points}\n")
print(response.text)

11329
In Q1 2021, Aaron's reported solid financial performance, with an 11.1% increase in revenue to $481.1 million. This growth was driven by strong in-store and online performance. Same-store revenue rose an impressive 14.8%, marking the first double-digit growth since 2009.

Aaron's made significant strides in its digital transformation. E-commerce now accounts for over 14% of lease revenues and grew 42% year-over-year, with optimized decision-making and enhanced payment platforms contributing to this success.

The company's lease portfolio quality also improved, with a 6.2% growth in size and reduced write-offs. Centralized decisioning and operational enhancements played a crucial role in this improvement.

Despite the closure of 166 stores, operational expenses (excluding restructuring costs) saw a modest decline. Adjusted EBITDA surged by 112.9% to $73.9 million, with an improved EBITDA margin of 15.4%.

Non-GAAP earnings per share soared by 313.3% to $1.24. The company is optimi